Notebook'un olduğu klasörde şu dosyalar olmalıdır:
data.csv = data,
word2DocFreq.json = Her kelime için TF-IDF'teki document frequency,
turkce_kelime_listesi.txt = Türkçe kelimelerin listesi

In [11]:
DOCUMENT_COUNT = 4000

In [9]:
import sys
import string

import zeyrek

analyzer = zeyrek.MorphAnalyzer()

def getLemmatizedWordList(sentence):
    return [x[0].lemma for x in analyzer.analyze(sentence) if len(x) > 0 and x[0].lemma not in [*string.punctuation, 'Unk']]

def getPOSTagList(sentence):
    return [x[0].pos for x in analyzer.analyze(sentence) if len(x) > 0 and x[0].pos not in ['Punc', 'Unk']]

In [30]:
import pandas as pd
import string
import json

TR_STOP = ["ya da", "yada", "a","acaba","altı","altmış","ama","ancak","arada","artık","asla","aslında","aslında","ayrıca","az","bana","bazen","bazı","bazıları","belki","ben","benden","beni","benim","beri","beş","bile","bilhassa","bin","bir","biraz","birçoğu","birçok","biri","birisi","birkaç","birşey","biz","bizden","bize","bizi","bizim","böyle","böylece","bu","buna","bunda","bundan","bunlar","bunları","bunların","bunu","bunun","burada","bütün","çoğu","çoğunu","çok","çünkü","da","daha","dahi","dan","de","defa","değil","diğer","diğeri","diğerleri","diye","doksan","dokuz","dolayı","dolayısıyla","dört","e","edecek","eden","ederek","edilecek","ediliyor","edilmesi","ediyor","eğer","elbette","elli","en","etmesi","etti","ettiği","ettiğini","fakat","falan","filan","gene","gereği","gerek","gibi","göre","hala","halde","halen","hangi","hangisi","hani","hatta","hem","henüz","hep","hepsi","her","herhangi","herkes","herkese","herkesi","herkesin","hiç","hiçbir","hiçbiri","i","ı","için","içinde","iki","ile","ilgili","ise","işte","itibaren","itibariyle","kaç","kadar","karşın","kendi","kendilerine","kendine","kendini","kendisi","kendisine","kendisini","kez","ki","kim","kime","kimi","kimin","kimisi","kimse","kırk","madem","mi","mı","milyar","milyon","mu","mü","nasıl","ne","neden","nedenle","nerde","nerede","nereye","neyse","niçin","nin","nın","niye","nun","nün","o","öbür","olan","olarak","oldu","olduğu","olduğunu","olduklarını","olmadı","olmadığı","olmak","olması","olmayan","olmaz","olsa","olsun","olup","olur","olur","olursa","oluyor","on","ön","ona","önce","ondan","onlar","onlara","onlardan","onları","onların","onu","onun","orada","öte","ötürü","otuz","öyle","oysa","pek","rağmen","sana","sanki","sanki","şayet","şekilde","sekiz","seksen","sen","senden","seni","senin","şey","şeyden","şeye","şeyi","şeyler","şimdi","siz","siz","sizden","sizden","size","sizi","sizi","sizin","sizin","sonra","şöyle","şu","şuna","şunları","şunu","ta","tabii","tam","tamam","tamamen","tarafından","trilyon","tüm","tümü","u","ü","üç","un","ün","üzere","var","vardı","ve","veya","ya","yani","yapacak","yapılan","yapılması","yapıyor","yapmak","yaptı","yaptığı","yaptığını","yaptıkları","ye","yedi","yerine","yetmiş","yi","yı","yine","yirmi","yoksa","yu","yüz","zaten","zira"]
TR_STOP = set(TR_STOP)

DATASET_PATH = r"data.csv"
dataDF = pd.read_csv(DATASET_PATH)
    
WORD_TO_DF_JSON_PATH = r"word2DocFreq.json"
with open(WORD_TO_DF_JSON_PATH, 'r', encoding="utf8") as word2FreqFile:
    WORD_TO_DF = json.load(word2FreqFile)

VOCABULARY_PATH = r"turkce_kelime_listesi.txt"
with open(VOCABULARY_PATH, 'r', encoding="utf8") as vocabFile:
    VOCABULARY = json.load(vocabFile, strict=False)

In [17]:
%pip install rake-nltk

Note: you may need to restart the kernel to use updated packages.
True


In [42]:
from rake_nltk import Rake
import re
import math

TEXT = dataDF.iloc[0]["data_text"]
LEMMATIZED_TEXT = ' '.join(getLemmatizedWordList(TEXT))

rake = Rake(language="turkish", max_length=6)

rake.extract_keywords_from_text(TEXT)

keywordsWithScores = list(set(rake.get_ranked_phrases_with_scores()))

finalKeywordsWithScores = []
for kwIndex in range(len(keywordsWithScores)):
    score, kw = keywordsWithScores[kwIndex]
    kw = kw.lower()
    if len(kw) >= 2 and not kw.isdigit():
        words = kw.split(' ')
        docFreq = 1
        OOVCount = 0
        for word in words:
            if not word in VOCABULARY:
                OOVCount += 1
            docFreq *= WORD_TO_DF.get(word, 1)
        docFreq /= len(words)
        
        # Out of vocabulary (OOV) filtrelemesi:
        if OOVCount / len(words) >= 0.6:
            continue
            
        lemmatizedWords = getLemmatizedWordList(kw)
        
        # TF-IDF etkisinin çarpılması:
        score *= len(re.findall(fr"\b{' '.join(lemmatizedWords)}\b", LEMMATIZED_TEXT)) * math.log(DOCUMENT_COUNT / docFreq, 10)
        
        finalKeywordsWithScores.append((kw, score))
        
print(sorted(finalKeywordsWithScores, reverse=True, key=lambda elem: elem[1])[:10])

[('ödenek', 52.18882790656838), ('gelir', 14.172133332659614), ('kurul', 13.29051335893929), ('hibe', 12.994691088312916), ('altyapı', 9.637254967214696), ('inşa', 8.87324368123922), ('kurum', 8.133166475355393), ('fonksiyonel', 8.089105578844608), ('kuruluş', 7.799895855642738), ('ikinci', 7.706870130578963)]
